In [1]:
import requests
import os
import pandas as pd
import gzip
import io
import numpy as np
import GEOparse

In [2]:
gse = GEOparse.get_GEO(geo="GSE25066", destdir="data/")

#build expression matrix (genes as rows, samples as columns, then transpose)
expression_data = pd.DataFrame({
    gsm_name: gsm.table.set_index("ID_REF")["VALUE"]
    for gsm_name, gsm in gse.gsms.items()
}).T

labels = {}

for gsm_name, gsm in gse.gsms.items():
    characteristics = gsm.metadata.get("characteristics_ch1", [])
    response_line = next((c for c in characteristics if "response" in c.lower() or "pcr" in c.lower()), None)

    if response_line:
        try:
            label_str = response_line.split(":")[1].strip().lower()
            if label_str == "pcr":
                labels[gsm_name] = 1
            elif label_str == "rd":
                labels[gsm_name] = 0
            else:
                labels[gsm_name] = None  #"na" or unexpected
        except Exception:
            labels[gsm_name] = None
    else:
        labels[gsm_name] = None

label_series = pd.Series(labels)

#merge labels with expression data
df = expression_data.merge(label_series.rename("pcr_response"), left_index=True, right_index=True)

#drop samples with missing expression or missing label
n_before = df.shape[0]
df_clean = df.dropna()
n_after = df_clean.shape[0]
df_clean["pcr_response"] = df_clean["pcr_response"].astype(int)

print(f"Samples before dropping missing: {n_before}")
print(f"Samples after dropping missing: {n_after}")
print(f"Samples discarded: {n_before - n_after}")

#show counts of pCR response labels
print(df_clean["pcr_response"].value_counts().sort_index())

26-Jun-2025 22:18:38 DEBUG utils - Directory data/ already exists. Skipping.
26-Jun-2025 22:18:38 INFO GEOparse - File already exist: using local version.
26-Jun-2025 22:18:38 INFO GEOparse - Parsing data/GSE25066_family.soft.gz: 
26-Jun-2025 22:18:38 DEBUG GEOparse - DATABASE: GeoMiame
26-Jun-2025 22:18:38 DEBUG GEOparse - SERIES: GSE25066
26-Jun-2025 22:18:38 DEBUG GEOparse - PLATFORM: GPL96


26-Jun-2025 22:18:44 DEBUG GEOparse - SAMPLE: GSM615096
26-Jun-2025 22:18:44 DEBUG GEOparse - SAMPLE: GSM615097
26-Jun-2025 22:18:44 DEBUG GEOparse - SAMPLE: GSM615098
26-Jun-2025 22:18:44 DEBUG GEOparse - SAMPLE: GSM615099
26-Jun-2025 22:18:45 DEBUG GEOparse - SAMPLE: GSM615100
26-Jun-2025 22:18:45 DEBUG GEOparse - SAMPLE: GSM615101
26-Jun-2025 22:18:45 DEBUG GEOparse - SAMPLE: GSM615102
26-Jun-2025 22:18:45 DEBUG GEOparse - SAMPLE: GSM615103
26-Jun-2025 22:18:45 DEBUG GEOparse - SAMPLE: GSM615104
26-Jun-2025 22:18:45 DEBUG GEOparse - SAMPLE: GSM615105
26-Jun-2025 22:18:46 DEBUG GEOparse - SAMPLE: GSM615106
26-Jun-2025 22:18:46 DEBUG GEOparse - SAMPLE: GSM615107
26-Jun-2025 22:18:46 DEBUG GEOparse - SAMPLE: GSM615108
26-Jun-2025 22:18:46 DEBUG GEOparse - SAMPLE: GSM615109
26-Jun-2025 22:18:46 DEBUG GEOparse - SAMPLE: GSM615110
26-Jun-2025 22:18:46 DEBUG GEOparse - SAMPLE: GSM615111
26-Jun-2025 22:18:46 DEBUG GEOparse - SAMPLE: GSM615112
26-Jun-2025 22:18:46 DEBUG GEOparse - SAMPLE: GS

Samples before dropping missing: 508
Samples after dropping missing: 488
Samples discarded: 20
pcr_response
0    389
1     99
Name: count, dtype: int64


/tmp/ipykernel_1753170/3548524297.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean["pcr_response"] = df_clean["pcr_response"].astype(int)


In [44]:
df_clean.to_csv("/omics/groups/OE0540/internal/users/de_pietri/code/project/data/df_final3.csv", index=True)